In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from scipy.special import expit
from PIL import Image
import requests

class TextClassifier:
    def __init__(self):
        MODEL = "cardiffnlp/tweet-topic-21-multi"
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL)
        self.model = AutoModelForSequenceClassification.from_pretrained(MODEL)
        self.class_mapping = self.model.config.id2label
    
    def classify(self, inputs):
        results = []
        for input in inputs:            
            tokens = self.tokenizer(input, return_tensors='pt')
            output = self.model(**tokens)
            scores = output[0][0].detach().numpy()
            scores = expit(scores)
            predictions = (scores >= 0.5) * 1
            labels = []
            for i in range(len(predictions)):
                if predictions[i]:
                    labels.append(self.class_mapping[i])
            results.append(labels)
        return results


In [ ]:
class ImageClassifier:
    def __init__(self):
        self.checkpoint = "openai/clip-vit-large-patch14"
        self.classifier = pipeline(model=self.checkpoint, task="zero-shot-image-classification")
        
    def classify(self, urls):
        results = []
        for url in urls:
            image = Image.open(requests.get(url, stream=True).raw)
            predictions = self.classifier(image, candidate_labels=["fox", "bear", "seagull", "owl"])
            results.append(predictions)
        return results


In [22]:
tc = TextClassifier()
ic = ImageClassifier()
print(tc.classify(["It is great to see athletes promoting awareness for climate change.", "I like burger king"]))
print(ic.classify(["https://unsplash.com/photos/g8oS8-82DxI/download?ixid=MnwxMjA3fDB8MXx0b3BpY3x8SnBnNktpZGwtSGt8fHx8fDJ8fDE2NzgxMDYwODc&force=true&w=640"]))

[['news_&_social_concern', 'sports'], ['food_&_dining']]
